In [2]:
%%sh

docker_name=`docker ps --format '{{.Names}}'`
#echo $docker_name
docker kill $docker_name

"docker kill" requires at least 1 argument.
See 'docker kill --help'.

Usage:  docker kill [OPTIONS] CONTAINER [CONTAINER...]

Kill one or more running containers


In [ ]:
!tree model_untarred

In [ ]:
!pygmentize inference.py

In [ ]:
!rm -rfv model_untarred/code/protobuf.tar.gz
!(cd model_untarred/code/ && tar -czvf protobuf.tar.gz __init__.py image_pb2.py)

In [ ]:
!(cd model_untarred/ && tar -czvf /tmp/model-resnet-152-inference.tar.gz *)

In [ ]:
!aws s3 cp /tmp/model-resnet-152-inference.tar.gz s3://ar54/super_models/
!aws s3 ls s3://ar54/super_models/   

In [ ]:
!aws s3 ls s3://ar54/super_models/model-resnet-152-inference.tar.gz

In [ ]:
import mxnet as mx
import mxnet.ndarray as nd
from mxnet import nd, autograd, gluon
from mxnet.gluon.data.vision import transforms

import numpy as np

model_arch_path = '/home/ec2-user/SageMaker/resnet-inference-model/model_untarred/resnet-152-symbol.json'
model_params_path = '/home/ec2-user/SageMaker/resnet-inference-model/model_untarred/resnet-152-0000.params'

# Use GPU if one exists, else use CPU
ctx = mx.gpu() if mx.context.num_gpus() else mx.cpu()
symbol = mx.sym.load(model_arch_path)

#mx.viz.print_summary(symbol)

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=chazarey-mxnet-serving-160-gpu-py3

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

#docker build -t chazarey-mxnet-serving:1.6.0-gpu-py3 -f docker/1.6.0/py3/Dockerfile.gpu .

docker build -t ${algorithm_name} -f Dockerfile.gpu .

docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
%%time

import sagemaker

role = sagemaker.session.get_execution_role()

from sagemaker.mxnet import MXNetModel

model_data="s3://ar54/super_models/model-resnet-152-inference.tar.gz"

model = MXNetModel(
    model_data=model_data,
    role=role,
    image="308412838853.dkr.ecr.us-east-2.amazonaws.com/chazarey-mxnet-serving-160-gpu-py3:latest",
    entry_point="inference.py",
    py_version='py3',
    framework_version='1.6.0'
)

#predictor = model.deploy(instance_type="local_gpu", initial_instance_count=1)
predictor = model.deploy(instance_type="local", initial_instance_count=1)

In [ ]:
!wget -O /tmp/test.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0007.jpg
file_name = '/tmp/test.jpg'
# test image
from IPython.display import Image
Image(file_name)  

In [ ]:
import io
import numpy as np
import json
from sagemaker.predictor import StringDeserializer
import time


def numpy_bytes_serializer(data):
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

predictor.serializer = None
predictor.deserializer = StringDeserializer()
predictor.accept = None
predictor.content_type = None

with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)

for i in range(0, 2):    
    response = predictor.predict(payload)            
    time.sleep(1)

#response

In [ ]:
#!sudo pip uninstall protobuf
#!sudo pip install -U protobuf
#!pip install -U protobuf

In [ ]:
import io
import numpy as np
import json
from sagemaker.predictor import StringDeserializer
import time
import image_pb2 as impb

with open('/tmp/test.jpg', 'rb') as f:
    payload = f.read()
    #payload = bytearray(payload)

image_packet = impb.PBImage()
image_packet.image_data = payload

def numpy_bytes_serializer(data):
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

predictor.serializer = None
predictor.deserializer = StringDeserializer()
predictor.accept = None
predictor.content_type = 'application/octet-stream'

for i in range(0, 2):    
    response = predictor.predict(image_packet.SerializeToString())            
    time.sleep(1)
    print(response)